In [ ]:
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTFigure
import pandas as pd
import numpy as np
import re

def parse_layout(layout, a, b):
    
    textDf = pd.DataFrame(columns=['x1', 'y1', 'x2', 'y2','y','text'])
    
    """Function to recursively parse the layout tree."""
    for lt_obj in layout:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            for obj in lt_obj:
                s = pd.Series({'x1':int(obj.bbox[0]), 'y1':int(obj.bbox[1]),'x2':int(obj.bbox[2]), 
                               'y2':int(obj.bbox[3]), 'y':float(0), 'text':obj.get_text().replace("\n", '')})
                textDf = textDf.append(s, ignore_index = True)  

    textDf = textDf.sort_values(['x1','y1'], ascending=True).reset_index(drop = True)
    
    # get the first characters' position of each row
    row_startDf = textDf[textDf['x1'] < textDf['x1'].min(axis=0) + b]
    
#     for i in range(len(textDf)):
#         print (textDf.iloc[i]['text'])
#         if 'End of course result history' in textDf.iloc[i]['text']:
#             print(i)
#             s = pd.Series({'x1':textDf.iloc[i]['x1'], 'y1':int(textDf.iloc[i]['y1']),'x2':int(textDf.iloc[i]['x2']), 
#                                'y2':int(textDf.iloc[i]['y2']), 'y':float(0), 'text':textDf.iloc[i]['text']})
#             row_startDf.append(s, ignore_index = True)
    
    rows = row_identifier(row_startDf, a)
    
    # check which row is each column in and set their y axis as the row's y
    for i in range(len(textDf)):
        y = -1
        for j in range(len(rows)):
            if float(textDf.iloc[i]['y2']) < rows.iloc[j]['upper_y'] and float(textDf.iloc[i]['y1']) > rows.iloc[j]['lower_y']:
                y = rows.iloc[j]['y']
                break
        #textDf.set_value(i, 'y', y)
        textDf.at[i:i, 'y'] = y
    
    return textDf

# set a wider boder for each row and set the middle position of row as y
def row_identifier(row_startDf, a):
    rows = pd.DataFrame(columns=['lower_y', 'upper_y', 'y'])
    height = row_startDf['y2']-row_startDf['y1']
    rows['upper_y'] = row_startDf['y2'] + height*a
    rows['lower_y'] = row_startDf['y1'] - height*a
    rows['y'] = (rows['upper_y'] + rows['lower_y'])/2
    return rows 
       

def convert_to_df(text):
    
        student = []
        course = []
        mark = []
        result = []
        previousR = []
        indexMark = -1
        indexPreviousS = -1
        indexResultE = -1
        
        table_start = 0
        table_end = 0
        
        for i in range(len(text)):
            content = text.iloc[i]['text']
            temp = content.split(',')
            if 'Student name' in temp[0]:
                student.append(temp[0][12:].lstrip())
            elif 'Student ID' in temp[0]:
                student.append(temp[0][10:].lstrip())
            elif 'Student course ID' in temp[0]:
                student.append(temp[0][17:].lstrip())
            elif temp[0] == 'Course':
                course.append(temp[1])
            elif len(temp) == 1 and 'Credit points obtained' in temp[0]:
                course.append(temp[0][22:])
            elif len(temp) == 2 and 'Credit points obtained' in temp[0]:
                course.append(temp[1])
            elif len(temp) == 2 and 'Major' in temp[0]:
                course.append(temp[1])
            elif 'Credit from previous study' in temp[0]:
                indexPreviousS = i + 2
            elif 'Credit points gained' in temp[0] and table_start == 0:
                indexResultE = i - 1
            elif 'Unit of Study Results' in temp[0]:
                table_start = i + 2
            elif 'Credit points gained' in temp[0] and table_start > 0:
                table_end = i - 1
            elif len(temp) == 4 and temp[0] == 'Year':
                indexMark = i
                break
                     
        
        if indexPreviousS != -1:
            print(indexResultE)
            for i in range(indexPreviousS, indexResultE):
                        content = text.iloc[i]['text']
                        temp = content.split(',')
                        print(temp)
                        if len(temp) < 3:
                            print("first row")
                            continue                            
                        previousR.append(temp)
            print (previousR)
        
        if table_end != 0:
            for i in range(table_start, table_end + 1):
                        content = text.iloc[i]['text']
                        temp = content.split(',')
                        
                        if 'Year' in temp[0]:
                            continue
                        temp0 = temp[0].split(' ')

                            
                        if (len(temp0) == 3):
                            temp.pop(0)
                            temp.insert(0, temp0[2])
                            temp.insert(0, temp0[1])
                            temp.insert(0, temp0[0])
                        elif len(temp0) == 2:
                            temp.pop(0)
                            temp.insert(0, temp0[1])
                            temp.insert(0, temp0[0])
                            
                        if len(temp) == 6:
                            temp1 = temp[4].split(' ')
                            if(len(temp1) == 1):
                                temp.insert(4, '')
                            elif len(temp1) == 2:
                                temp.pop(4)
                                temp.insert(4, temp1[1])
                                temp.insert(4, temp1[0])
                            
                        result.append(temp)
        else:
            for i in range(table_start, len(text)):
                        content = text.iloc[i]['text']
                        temp = content.split(',')
                        if 'Year' in temp[0]:
                            continue
                        temp0 = temp[0].split(' ')
                            
                        if (len(temp0) == 3):
                            temp.pop(0)
                            temp.insert(0, temp0[2])
                            temp.insert(0, temp0[1])
                            temp.insert(0, temp0[0])
                        elif len(temp0) == 2:
                            temp.pop(0)
                            temp.insert(0, temp0[1])
                            temp.insert(0, temp0[0])
                            
                        if len(temp) == 6:
                            temp1 = temp[4].split(' ')
                            if(len(temp1) == 1):
                                temp.insert(4, '')
                            elif len(temp1) == 2:
                                temp.pop(4)
                                temp.insert(4, temp1[1])
                                temp.insert(4, temp1[0])
                            
                        result.append(temp)

        if text.iloc[len(text)-1]['text'] == 'End of course result history':        
            #print("end")
            for i in range(indexMark, len(text)-1):
                content = text.iloc[i]['text']
                temp = content.split(',')
                if 'Year' in temp[0]:
                            continue
                mark.append(temp)
            mark = [y for x in mark for y in x]
            mark = np.reshape(mark, (-1, 4))
        elif indexMark > -1:
            #print("no end")
            for i in range(indexMark+1, len(text)):
                content = text.iloc[i]['text']
                temp = content.split(',')
                mark.append(temp)
    
        #s = pd.Series({'Student name':student[0], 'Student ID':student[1], 'Student course ID':student[2]})
        studentDf = pd.DataFrame(columns = ('Student name', 'Student ID', 'Student course ID'), data = [student])
        if len(course) == 3:
            courseDf = pd.DataFrame(columns = ('Course', 'Credit points obtained', 'Major(s)'), data = [course])
        elif len(course) == 2:
            courseDf = pd.DataFrame(columns = ('Course', 'Credit points obtained'), data = [course])
        marksDf = pd.DataFrame(columns = ('Year', 'Session', 'Semester average mark', 'Annual average mark'), data = mark)
        resultDf = pd.DataFrame(columns = ('Year', 'Session', 'Unit of study code', 'Unit of study name', 'Mark', 'Grade', 'Credit points'), data = result)
        previousDf = pd.DataFrame(columns = ('Unit of study Code', 'Unit of study Name', 'Credit points'), data = previousR)
    
        return studentDf, courseDf, marksDf, resultDf, previousDf
    

In [ ]:
def convert_to_df2(text):
    result = []
    mark = []
    markSum = []
    indexResult = 0
    indexMark = 0
    for i in range(len(text)-1):
        content = text.iloc[i]['text']
        temp = content.split(',')
        if 'Credit points gained' in content:
            indexResult = i+1
            break
        elif temp[0] == 'Year':
#             print("2")
            indexMark = i+1
            break
    
    if indexResult > 0:
        if indexResult != 1:
            for i in range(0,indexResult-1): 
                content = text.iloc[i]['text']
                temp = content.split(',')
                if 'Year' in temp[0]:
                    continue
                temp0 = temp[0].split(' ')
                    
                if (len(temp0) == 3):
                    temp.pop(0)
                    temp.insert(0, temp0[2])
                    temp.insert(0, temp0[1])
                    temp.insert(0, temp0[0])
                elif len(temp0) == 2:
                    temp.pop(0)
                    temp.insert(0, temp0[1])
                    temp.insert(0, temp0[0])
                    
                if len(temp) == 6:
                    temp1 = temp[4].split(' ')
                    if(len(temp1) == 1):
                        temp.insert(4, '')
                    elif len(temp1) == 2:
                        temp.pop(4)
                        temp.insert(4, temp1[1])
                        temp.insert(4, temp1[0])
                    
                result.append(temp)

        for i in range(indexResult+2, len(text)-1):
            content = text.iloc[i]['text']
            temp = content.split(',')
            mark.append(temp)

    if indexMark > 0 :
        
        for i in range(indexMark, len(text)-1):
            content = text.iloc[i]['text']
            temp = content.split(',')
            mark.append(temp)
        
    elif indexMark == 0 and indexResult == 0:
        for i in range(len(text)-1):
            content = text.iloc[i]['text']
            temp = content.split(',')  #['','','','']
            mark.append(temp)           
                
    resultDf = pd.DataFrame(columns = ('Year', 'Session', 'Unit of study code', 'Unit of study name', 'Mark', 'Grade', 'Credit points'), data = result)
    marksDf2 = pd.DataFrame(columns = ('Year', 'Session', 'Semester average mark', 'Annual average mark'), data = mark)
        
    return resultDf, marksDf2
    

In [ ]:

def convert_pdf_to_df(file):
    
    fp = open(file, 'rb')
    parser = PDFParser(fp)
    doc = PDFDocument(parser)
    
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    textDf = pd.DataFrame(columns=['x1', 'y1', 'x2', 'y2', 'text'])
    resultDf = pd.DataFrame(columns = ('Year', 'Session', 'Unit of study code', 'Unit of study name', 'Mark', 'Grade', 'Credit points'))
    marksDf2 = pd.DataFrame(columns = ('Year', 'Session', 'Semester average mark', 'Annual average mark'))
    resultDf2 = pd.DataFrame(columns = ('Year', 'Session', 'Unit of study code', 'Unit of study name', 'Mark', 'Grade', 'Credit points'))
    
    page_no = 1
    
    for page in PDFPage.create_pages(doc):
        
        interpreter.process_page(page)
        layout = device.get_result()
        textDf = parse_layout(layout, 0.3, 30)
        textDf['text'] = textDf['text'].apply(lambda x:','+ x)
        textDf1 = textDf.groupby(by='y')['text'].sum().reset_index()
        textDf1 = textDf1.sort_values('y', ascending=False).reset_index(drop=True)
        textDf1['text'] = textDf1['text'].apply(lambda x:x[1:])
        #print(textDf1)
    
        if page_no == 1:
            studentDf, courseDf, marksDf, resultDf1, previousDf = convert_to_df(textDf1)
        elif page_no == 2:
            resultDf2, marksDf2 = convert_to_df2(textDf1)
            
        page_no += 1
        
    
    if marksDf2.empty == False:
        marksDf = pd.concat([marksDf, marksDf2])
    resultDf = pd.concat([resultDf1, resultDf2])
    
    return studentDf, courseDf, marksDf, resultDf, previousDf
    
    

In [ ]:
studentDf, courseDf, marksDf, resultDf, previouDf = convert_pdf_to_df("pdfFiles/Transcript0903.pdf")

In [5]:
studentDf

,Student name,Student ID,Student course ID
0,Shiyu Wang,450271782,450271782/1


In [162]:
courseDf

,Course,Credit points obtained
0,Bachelor of Engineering Honours,192


In [163]:
resultDf

,Year,Session,Unit of study code,Unit of study name,Mark,Grade,Credit points
0,2015,S1C,ELEC1103,Fundamentals of Elec and Electronic Eng,96.0,HD,6
1,2015,S1C,ENGG1801,Engineering Computing,93.0,HD,6
2,2015,S1C,MATH1001,Differential Calculus,86.0,HD,3
3,2015,S1C,MATH1002,Linear Algebra,88.0,HD,3
4,2015,S1C,MTRX1701,Mechatronic Engineering Introductory,75.0,DI,6
5,2015,S2C,AMME1362,Materials 1,74.0,CR,6
6,2015,S2C,ENGG1802,Engineering Mechanics,84.0,DI,6
7,2015,S2C,MATH1003,Integral Calculus and Modelling,73.0,CR,3
8,2015,S2C,MATH1005,Statistics,85.0,HD,3
9,2015,S2C,MTRX1702,Mechatronics 1,81.0,DI,6


In [164]:
marksDf

,Year,Session,Semester average mark,Annual average mark
0,2015,Semester 1,87.8,NA
1,2015,Semester 2,79.5,83.6
2,2016,Semester 1,80.3,NA
3,2016,Semester 2,86.3,83.3
4,2017,Semester 1,84.0,NA
5,2017,Semester 2,87.8,85.2
6,2018,Semester 1,84.0,NA
7,2018,Semester 2,80.7,82.3
8,2019,Semester 1,82.5,NA


In [165]:
previouDf

,Unit of study Code,Unit of study Name,Credit points
